<a href="https://colab.research.google.com/github/minkyeongk/DL_TensorFlow_Developer/blob/main/3.%20Natural%20Language%20Processing%20in%20TensorFlow/week_2/week_2_notebook_1_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#word embedding 

In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJR551O/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJR551O/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJR551O/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels =[]

test_sentences = []
test_labels = []

for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

# 받아온 데이터들을 실제 데이터와 레이블로 나누어 저장 
# 데이터 불러오고 이렇게 나누는데도 한참이 걸리는구나 

In [8]:
vocab_size = 10000
embedding_dim = 16
max_len = 120 
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen = max_len, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_len, truncating=trunc_type)

In [10]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(train_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [12]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
                             tf.keras.layers.Flatten(),    # 대신 다른 코드 사용되기도 함 
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 10
model.fit(padded, train_labels_final, epochs=num_epochs, validation_data=(test_padded, test_labels_final))
# 전에 ImageDataGenerator 사용법 복습하기 
# overfitting 발생 

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3656 - accuracy: 0.8414 - val_loss: 0.3762 - val_accuracy: 0.8312
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1870 - accuracy: 0.9330 - val_loss: 0.4244 - val_accuracy: 0.8211
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0642 - accuracy: 0.9861 - val_loss: 0.5335 - val_accuracy: 0.8099
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0153 - accuracy: 0.9985 - val_loss: 0.6176 - val_accuracy: 0.8102
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0043 - accuracy: 0.9997 - val_loss: 0.6904 - val_accuracy: 0.8097
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.7457 - val_accuracy: 0.8112
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 7.4235e-04 - accuracy: 1.0000 - val_loss: 0.7977 - val_accuracy: 0.8112
Ep

In [16]:
e = model.layers[0]   # embedding layer
weights = e.get_weights()[0]
print(weights.shape)  # shape: (vocab_size, embedding_dim)

(10000, 16)


In [17]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
